In [1]:
"""
This script reads a list of molecules [(SMILES, XYZ), ...] from a file,
performs MMFF94 force-field optimization, and writes the lowest-energy
molecule to an output file in the same format:
[(SMILES_LOWEST_ENERGY, XYZ_LOWEST_ENERGY)]
"""

import ast
from openbabel import openbabel as ob

def find_lowest_energy_molecule(candidates, steps=2000, crit=1e-5):
    """
    Finds the lowest-energy molecule from a list of (SMILES, XYZ) tuples
    using MMFF94 force-field energy minimization.

    Parameters
    ----------
    candidates : list of tuples
        List of (SMILES, XYZ_string) molecules.
    steps : int
        Maximum number of conjugate gradient steps.
    crit : float
        Convergence criterion for energy minimization.

    Returns
    -------
    list
        Single-element list containing (SMILES_lowest, XYZ_lowest).
    """

    def xyz_to_obmol(xyz):
        """Convert XYZ string to OpenBabel molecule object."""
        conv = ob.OBConversion()
        conv.SetInFormat("xyz")
        mol = ob.OBMol()
        if not conv.ReadString(mol, xyz.strip()):
            raise ValueError("Failed to parse XYZ string.")
        return mol

    def obmol_to_xyz(mol):
        """Convert OpenBabel molecule object to XYZ string."""
        conv = ob.OBConversion()
        conv.SetOutFormat("xyz")
        return conv.WriteString(mol)

    best_energy = None
    best_entry = None

    for smi, xyz in candidates:
        try:
            mol = xyz_to_obmol(xyz)
            mol.AddHydrogens()  # Ensure all hydrogens are present

            ff = ob.OBForceField.FindForceField("MMFF94")
            if not ff or not ff.Setup(mol):
                print(f"⚠️ Skipping {smi}: MMFF94 not available or setup failed.")
                continue

            # Perform geometry optimization
            ff.ConjugateGradients(steps, crit)
            ff.GetCoordinates(mol)
            energy = ff.Energy()

            xyz_opt = obmol_to_xyz(mol)

            print(f"SMILES: {smi}, Energy = {energy:.4f} kcal/mol")

            # Update lowest-energy entry
            if best_energy is None or energy < best_energy:
                best_energy = energy
                best_entry = (smi, xyz_opt)

        except Exception as e:
            print(f"⚠️ Skipping {smi}: {e}")
            continue

    if best_entry is None:
        raise RuntimeError("No molecule could be minimized successfully.")

    return [best_entry]

In [2]:
# === Main Script ===
if __name__ == "__main__":
    # Read candidates from input file
    input_file = "molecules_input.txt"
    output_file = "lowest_energy_output.txt"

    with open(input_file, "r") as f:
        candidates = ast.literal_eval(f.read())

    # Find the lowest-energy molecule
    lowest = find_lowest_energy_molecule(candidates)

    # Write output to file
    with open(output_file, "w") as f:
        f.write(str(lowest))

    print(f"Lowest-energy molecule written to '{output_file}'")


SMILES: CCO, Energy = 12.4620 kcal/mol
SMILES: COC, Energy = 5.1141 kcal/mol
Lowest-energy molecule written to 'lowest_energy_output.txt'
